In [32]:
import pandas as pd
import os
current_dir = os.getcwd()
current_dir
os.chdir(os.path.dirname(current_dir))
print("You set a new current directory")
current_dir = os.getcwd()
current_dir
#set current working directory to H:\VScode\March Group\March_Team_Project\
os.chdir("H:\\VScode\\March Group\\March_Team_Project\\")

You set a new current directory


In [33]:
# Issue 16 and Issue 15
# Merge weather data with pollution data, ensure pollution data is filtered for locations and timeperiod.
# Merge on date and location
# Save the merged data to a new csv file
# load pollution data
pollution_data = pd.read_csv('Source_Data\\filtered_pollution.zip')
# count of records in pollution data
print(pollution_data.shape)
# filter pollution data for locations and timeperiod
pollution_filter = pd.read_csv('Outputs/pollution_data_available.csv')
print(pollution_filter.shape)
# drop rows with missing values
pollution_data = pollution_data.dropna()
# count of records in pollution filter
print(pollution_data.shape)
# filter pollution_data for location in polution_filter with date between years 2003 and 2016
pollution_data = pollution_data[pollution_data['Date Local'].str.contains('200[3-9]|201[0-6]')]
print(pollution_data.shape)
pollution_data = pollution_data[pollution_data['State'].isin(pollution_filter['State'])]
print(pollution_data.shape)
pollution_data
# drop Unnamed: 0 column
pollution_data = pollution_data.drop(columns=['Unnamed: 0'])
# summarize pollution data
pollution_data_summary = pollution_data.groupby(['State Code',
                                                 'County Code',
                                                 'Site Num',
                                                 'Address',
                                                 'State',
                                                 'County',
                                                 'City',
                                                 'Date Local',
                                                 'Year']).agg(
                                                     {'NO2 Mean': 'mean',
                                                      'NO2 1st Max Value': 'max',
                                                      'NO2 1st Max Hour': 'max',
                                                      'NO2 AQI': 'mean',
                                                      'O3 Mean': 'mean',
                                                      'O3 1st Max Value': 'max',
                                                      'O3 1st Max Hour': 'max',
                                                      'O3 AQI': 'mean',
                                                      'SO2 Mean': 'mean',
                                                      'SO2 1st Max Value': 'max',
                                                      'SO2 1st Max Hour': 'max',
                                                      'SO2 AQI': 'mean',
                                                      'CO Mean': 'mean',
                                                      'CO 1st Max Value': 'max',
                                                      'CO 1st Max Hour': 'max',
                                                      'CO AQI': 'mean'
                                                     }).reset_index()
# summarize pollution data
print(pollution_data_summary.shape)


(878851, 30)
(29, 6)
(219827, 30)
(219827, 30)
(219827, 30)
(213308, 25)


In [34]:
# provide a count of records by year
pollution_data['year'] = pollution_data['Date Local'].str[:4]
pollution_data.groupby('year').size()   


year
2003    18995
2004    18406
2005    19553
2006    20097
2007    22284
2008    23203
2009    19668
2010    18131
2011    19824
2012    19029
2013    20637
dtype: int64

In [36]:
# Create a cleaned version of pollution data where any row with a null value is removed
pollution_data_cleaned = pollution_data.dropna()
print(f"Shape after dropping null values: {pollution_data_cleaned.shape}")

# Format 'City' column as uppercase for consistency
pollution_data_cleaned['City'] = pollution_data_cleaned['City'].str.upper()

# Rename 'Date Local' to 'Date' for better readability and consistency
pollution_data_cleaned.rename(columns={'Date Local': 'Date'}, inplace=True)

# Convert 'Date' column to datetime format for easier date manipulation
pollution_data_cleaned['Date'] = pd.to_datetime(pollution_data_cleaned['Date'])

# Display the first few rows of the cleaned data for verification
pollution_data_cleaned.head()

Shape after dropping null values: (219827, 30)


,State Code,County Code,Site Num,Address,State,County,City,Date,NO2 Units,NO2 Mean,...,SO2 1st Max Value,SO2 1st Max Hour,SO2 AQI,CO Units,CO Mean,CO 1st Max Value,CO 1st Max Hour,CO AQI,Year,year
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,10.0,0,14.0,Parts per million,2.089474,4.6,5,52.0,2003,2003
5,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,10.0,6,14.0,Parts per million,1.737500,2.6,8,30.0,2003,2003
9,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-03,Parts per billion,42.291667,...,14.0,21,20.0,Parts per million,1.412500,2.9,23,33.0,2003,2003
13,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-04,Parts per billion,45.708333,...,13.0,1,19.0,Parts per million,2.275000,3.6,2,41.0,2003,2003
17,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-05,Parts per billion,32.958333,...,8.0,0,11.0,Parts per million,1.612500,3.1,0,35.0,2003,2003


In [37]:
# load weather data on day and city
weather_data = pd.read_csv('Outputs/Us_Weather_Final_10km_V2.zip')
# rename columns to match stations
weather_data = weather_data.rename(columns={'Station_ID':'StationId'})
# convert date to datetime current format yyyymmdd
weather_data['Date'] = weather_data['Date'].astype(str)
weather_data['Date'] = weather_data['Date'].apply(lambda x: x[:4] + '-' + x[4:6] + '-' + x[6:])
weather_data['Date'] = pd.to_datetime(weather_data['Date'])
# drop columns not needed
weather_data = weather_data.drop(columns=["Unnamed: 0", "Unnamed: 0.1"])
print(weather_data.shape)
weather_data

(384067, 23)


,StationId,Date,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,TSUN,WDMV,...,WT04,WT05,WT06,WT08,WT09,WT11,WT13,WT16,WT18,WT22
0,USC00010764,2003-01-01,NaN,NaN,36.0,NaN,144.0,39.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,USC00010764,2003-01-02,NaN,NaN,30.0,NaN,106.0,28.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,USC00010764,2003-01-03,NaN,NaN,0.0,NaN,39.0,-11.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,USC00010764,2003-01-04,NaN,NaN,0.0,NaN,111.0,-44.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,USC00010764,2003-01-05,NaN,NaN,0.0,NaN,167.0,22.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
384062,USW00094789,2013-12-27,49.0,NaN,0.0,21.0,56.0,-5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384063,USW00094789,2013-12-28,54.0,NaN,0.0,48.0,117.0,-5.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384064,USW00094789,2013-12-29,45.0,NaN,300.0,66.0,94.0,33.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
384065,USW00094789,2013-12-30,60.0,NaN,0.0,50.0,83.0,-43.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
# load Us_Stations_with_City_10km.csv
stations = pd.read_csv('Outputs/Us_Stations_with_City_10km.csv')
# get siteId from StationId - last 4 characters
stations['SiteId'] = stations['StationId'].str[-8:]
# format StationId as capitals
stations['StationId'] = stations['StationId'].str.upper()
# format city as capitals
stations['City'] = stations['City'].str.upper()
print(stations.shape)
stations


(607, 9)


,StationId,Latitude,Longitude,Elevation,Name,CountryCode,City,CityDistance,SiteId
0,US10CEDA014,42.3951,-97.0319,438.0,LAUREL 3.9 ESE,US,CONCORD,3.736847,0ceda014
1,US10DIXO001,42.4273,-96.9774,463.0,WAYNE 13.7 NNE,US,CONCORD,4.918946,0dixo001
2,US10DOUG025,41.3639,-96.1558,352.3,BENNINGTON 0.1 ESE,US,WASHINGTON,5.707573,0doug025
3,US10DOUG051,41.3553,-96.2138,341.1,BENNINGTON 3.0 W,US,WASHINGTON,4.733762,0doug051
4,US10DOUG054,41.3121,-96.1779,341.4,OMAHA 13.1 WNW,US,WASHINGTON,9.820842,0doug054
...,...,...,...,...,...,...,...,...,...
602,USW00093874,30.4122,-89.0808,4.9,GULFPORT - BILOXI AP,US,LONG BEACH,9.902063,00093874
603,USW00093928,32.7667,-96.7833,195.1,DALLAS WBO,US,DALLAS,3.359899,00093928
604,USW00093972,39.1000,-94.5833,294.1,KANSAS CITY,US,KANSAS CITY,3.653845,00093972
605,USW00094701,42.3500,-71.0667,6.1,BOSTON CITY WSO,US,BOSTON,3.787921,00094701


In [39]:
# merge weather data with stations on station id
weather_data = pd.merge(weather_data, stations, on='StationId')
print(weather_data.shape)
# set all null values to 0
weather_data = weather_data.fillna(0)
# rename columns Latitude and Longitude to WS_Latitude and WS_Longitude
weather_data = weather_data.rename(columns={'Latitude':'WS_Latitude', 'Longitude':'WS_Longitude', "Elevation": "WS_Elevation", "Name": "WS_Name"})
# filter to only keep 1 record per city per day based on the closest station
print(weather_data.shape)

weather_data

(380584, 31)
(380584, 31)


,StationId,Date,AWND,PGTM,PRCP,TAVG,TMAX,TMIN,TSUN,WDMV,...,WT18,WT22,WS_Latitude,WS_Longitude,WS_Elevation,WS_Name,CountryCode,City,CityDistance,SiteId
0,USC00010764,2003-01-01,0.0,0.0,36.0,0.0,144.0,39.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
1,USC00010764,2003-01-02,0.0,0.0,30.0,0.0,106.0,28.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
2,USC00010764,2003-01-03,0.0,0.0,0.0,0.0,39.0,-11.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
3,USC00010764,2003-01-04,0.0,0.0,0.0,0.0,111.0,-44.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
4,USC00010764,2003-01-05,0.0,0.0,0.0,0.0,167.0,22.0,0.0,0.0,...,0.0,0.0,33.3936,-87.0067,132.6,BESSEMER 3 WSW,US,CONCORD,8.742848,00010764
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
380579,US1VAAP0001,2013-12-23,0.0,0.0,244.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380580,US1VAAP0001,2013-12-24,0.0,0.0,64.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380581,US1VAAP0001,2013-12-29,0.0,0.0,163.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001
380582,US1VAAP0001,2013-12-30,0.0,0.0,157.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,37.4083,-78.9638,219.5,CONCORD 4.6 NNE,US,CONCORD,7.568900,VAAP0001


In [ ]:
# filter pollution data to have 1 record per city per day using the record with the lowest CityDistance
print(pollution_data_cleaned.shape)
# merge pollution data with weather data on date and location only take first match
merged_data = pd.merge(pollution_data_cleaned, weather_data, on=['Date', 'City'], how='inner')


# merged_data = pd.merge(pollution_data_cleaned, weather_data, left_on=['Date', 'City'], right_on=['Date', 'City'], how='inner')

print(merged_data.shape)
merged_data

(219827, 30)
(393645, 59)


,State Code,County Code,Site Num,Address,State,County,City,Date,NO2 Units,NO2 Mean,...,WT16,WT18,WT22,WS_Latitude,WS_Longitude,WS_Elevation,WS_Name,CountryCode,CityDistance,SiteId
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,0.0,0.0,0.0,43.1500,-76.3333,115.5,BALDWINSVILLE,US,9.561566,00300379
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,0.0,0.0,0.0,43.2386,-76.1964,114.9,BREWERTON LOCK 23,US,8.114212,00300870
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,0.0,0.0,0.0,43.1500,-76.3333,115.5,BALDWINSVILLE,US,9.561566,00300379
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,0.0,0.0,0.0,43.2386,-76.1964,114.9,BREWERTON LOCK 23,US,8.114212,00300870
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-03,Parts per billion,42.291667,...,0.0,0.0,0.0,43.1500,-76.3333,115.5,BALDWINSVILLE,US,9.561566,00300379
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393640,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-05-23,Parts per billion,22.876923,...,0.0,0.0,0.0,40.8092,-89.0546,219.8,EL PASO 5.2 NNW,US,8.227938,ILWF0014
393641,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-07,Parts per billion,5.185000,...,0.0,0.0,0.0,40.8092,-89.0546,219.8,EL PASO 5.2 NNW,US,8.227938,ILWF0014
393642,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-08,Parts per billion,9.520000,...,0.0,0.0,0.0,40.8092,-89.0546,219.8,EL PASO 5.2 NNW,US,8.227938,ILWF0014
393643,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-09,Parts per billion,9.282609,...,0.0,0.0,0.0,40.8092,-89.0546,219.8,EL PASO 5.2 NNW,US,8.227938,ILWF0014


In [49]:
# count of records By Date
datecount = merged_data.groupby('Date').size()
datecount

Date
2003-01-01     52
2003-01-02     52
2003-01-03     52
2003-01-04     52
2003-01-05     52
             ... 
2013-12-27    143
2013-12-28    134
2013-12-29    143
2013-12-30    145
2013-12-31    181
Length: 4018, dtype: int64

In [50]:
# count unique pollution sites per day
pollution_sites = merged_data.groupby('Date')['Site Num'].nunique()
pollution_sites

Date
2003-01-01    11
2003-01-02    11
2003-01-03    11
2003-01-04    11
2003-01-05    11
              ..
2013-12-27    17
2013-12-28    16
2013-12-29    16
2013-12-30    16
2013-12-31    17
Name: Site Num, Length: 4018, dtype: int64

In [ ]:
# deduplicate merged data to provide 1 row per Date, city, county, state, address, site num, county code, state code, choosing record with lowest CityDistance if multiple records exist
merged_data_sort = merged_data.sort_values(by=['Date','State', 'Address', 'County', 'Site Num', 'City', 'CityDistance']).copy()
print(merged_data_sort.shape)
merged_data_dedupe = merged_data_sort.drop_duplicates(subset=['Date', 'City', 'County', 'State', 'Address', 'Site Num', 'County Code', 'State Code'], keep='first')
print(merged_data_dedupe.shape)


(393645, 71)
(54114, 71)


In [43]:
#  display Percentage of null values in each column
null_values_percentage = merged_data_dedupe.isnull().sum() / len(merged_data) * 100
null_values_percentage


State Code           0.0
County Code          0.0
Site Num             0.0
Address              0.0
State                0.0
County               0.0
City                 0.0
Date                 0.0
NO2 Units            0.0
NO2 Mean             0.0
NO2 1st Max Value    0.0
NO2 1st Max Hour     0.0
NO2 AQI              0.0
O3 Units             0.0
O3 Mean              0.0
O3 1st Max Value     0.0
O3 1st Max Hour      0.0
O3 AQI               0.0
SO2 Units            0.0
SO2 Mean             0.0
SO2 1st Max Value    0.0
SO2 1st Max Hour     0.0
SO2 AQI              0.0
CO Units             0.0
CO Mean              0.0
CO 1st Max Value     0.0
CO 1st Max Hour      0.0
CO AQI               0.0
Year                 0.0
year                 0.0
StationId            0.0
AWND                 0.0
PGTM                 0.0
PRCP                 0.0
TAVG                 0.0
TMAX                 0.0
TMIN                 0.0
TSUN                 0.0
WDMV                 0.0
WT01                 0.0


In [44]:
# Apply AQI classifications to the data
# AQI = 0-50 Good, 51-100 Moderate, 101-150 Unhealthy for Sensitive Groups, 151-200 Unhealthy, 201-300 Very Unhealthy, 301-500 Hazardous
# Create a new column AQI_Range
# Create a new column AQI_Range_Description

def AQI_Range(value):
    if value >= 0 and value <= 50:
        return 'Good'
    if value >= 51 and value <= 100:
        return 'Moderate'
    if value >= 101 and value <= 150:
        return 'Unhealthy for Sensitive Groups'
    if value >= 151 and value <= 200:
        return 'Unhealthy'
    if value >= 201 and value <= 300:
        return 'Very Unhealthy'
    if value >= 301 and value <= 500:
        return 'Hazardous'
    return 'Unknown'

def AQI_Range_Description(value):
    if value >= 0 and value <= 50:
        return 'Air quality is considered satisfactory, and air pollution poses little or no risk.'
    if value >= 51 and value <= 100:
        return 'Air quality is acceptable; however, for some pollutants there may be a moderate health concern for a very small number of people who are unusually sensitive to air pollution.'
    if value >= 101 and value <= 150:
        return 'Members of sensitive groups may experience health effects. The general public is not likely to be affected.'
    if value >= 151 and value <= 200:
        return 'Everyone may begin to experience health effects; members of sensitive groups may experience more serious health effects.'
    if value >= 201 and value <= 300:
        return 'Health alert: everyone may experience more serious health effects.'
    if value >= 301 and value <= 500:
        return 'Health warnings of emergency conditions. The entire population is more likely to be affected.'
    return 'Unknown'

def AQI_Group(value):
    if value >= 0 and value <= 50:
        return 1
    if value >= 51 and value <= 100:
        return 2
    if value >= 101 and value <= 150:
        return 3
    if value >= 151 and value <= 200:
        return 4
    if value >= 201 and value <= 300:
        return 5
    if value >= 301 and value <= 500:
        return 6
    return 'Unknown'

# apply AQI_Range, AQI_Group and AQI_Range_Description to O3_AQI, SO2_AQI, CO_AQI, NO2_AQI
merged_data['O3_AQI_Range'] = merged_data['O3 AQI'].apply(AQI_Range)
merged_data['O3_AQI_Range_Description'] = merged_data['O3 AQI'].apply(AQI_Range_Description)
merged_data['O3_AQI_Group'] = merged_data['O3 AQI'].apply(AQI_Group)
merged_data['SO2_AQI_Range'] = merged_data['SO2 AQI'].apply(AQI_Range)
merged_data['SO2_AQI_Range_Description'] = merged_data['SO2 AQI'].apply(AQI_Range_Description)
merged_data['SO2_AQI_Group'] = merged_data['SO2 AQI'].apply(AQI_Group)
merged_data['CO_AQI_Range'] = merged_data['CO AQI'].apply(AQI_Range)
merged_data['CO_AQI_Range_Description'] = merged_data['CO AQI'].apply(AQI_Range_Description)
merged_data['CO_AQI_Group'] = merged_data['CO AQI'].apply(AQI_Group)
merged_data['NO2_AQI_Range'] = merged_data['NO2 AQI'].apply(AQI_Range)
merged_data['NO2_AQI_Range_Description'] = merged_data['NO2 AQI'].apply(AQI_Range_Description)
merged_data['NO2_AQI_Group'] = merged_data['NO2 AQI'].apply(AQI_Group)

merged_data

,State Code,County Code,Site Num,Address,State,County,City,Date,NO2 Units,NO2 Mean,...,O3_AQI_Group,SO2_AQI_Range,SO2_AQI_Range_Description,SO2_AQI_Group,CO_AQI_Range,CO_AQI_Range_Description,CO_AQI_Group,NO2_AQI_Range,NO2_AQI_Range_Description,NO2_AQI_Group
0,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2,Moderate,"Air quality is acceptable; however, for some p...",2
1,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-01,Parts per billion,40.000000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2,Moderate,"Air quality is acceptable; however, for some p...",2
2,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2
3,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-02,Parts per billion,40.000000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2
4,4,13,3002,1645 E ROOSEVELT ST-CENTRAL PHOENIX STN,Arizona,Maricopa,PHOENIX,2003-01-03,Parts per billion,42.291667,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Moderate,"Air quality is acceptable; however, for some p...",2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
393640,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-05-23,Parts per billion,22.876923,...,2,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
393641,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-07,Parts per billion,5.185000,...,2,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
393642,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-08,Parts per billion,9.520000,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1
393643,48,141,44,800 S San Marcial Street,Texas,El Paso,EL PASO,2013-07-09,Parts per billion,9.282609,...,1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1,Good,"Air quality is considered satisfactory, and ai...",1


In [46]:
# Count records per city - 
# Group by city and count records
city_count = merged_data.groupby(['State', 'City']).size()


expected = 365*10

city_count

State                 City        
Arizona               PHOENIX          31801
California            CONCORD          40008
                      LONG BEACH       32550
                      LOS ANGELES       7668
District Of Columbia  WASHINGTON      112266
Florida               WINTER PARK       5803
Kansas                KANSAS CITY       6488
Louisiana             BATON ROUGE      27803
Massachusetts         BOSTON           34195
New Jersey            CAMDEN           36408
New York              NEW YORK          1861
Pennsylvania          PHILADELPHIA     28103
Texas                 DALLAS           27606
                      EL PASO           1085
dtype: int64

In [ ]:
# Save merged to zip file
merged_data.to_csv('Outputs/Merged_Data_PG_1.zip', index=False)

In [ ]:
# convert "State Code", "County Code", "Site Num" to object
merged_data[["State Code", "County Code", "Site Num"]] = merged_data[["State Code", "County Code", "Site Num"]].astype(str)

# produce list of numeric and bool columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64', 'bool']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

print(numeric_columns)
# produce correlation matrix of merged_data for columns in numeric_columns
correlation_matrix = merged_data[numeric_columns].corr()


In [ ]:
# round all values in correlation matrix to 2 decimal places
correlation_matrix = correlation_matrix.round(2)
correlation_matrix

In [ ]:
# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
import seaborn as sns
import matplotlib.pyplot as plt
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix, annot=True, cmap='coolwarm', fmt=".1f")

plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Use scikit learn to apply transformations to the data to standardise the data and correct skewness
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import PowerTransformer
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

# create a list of numeric columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

# create a list of columns with skewness greater than 1
skewness = merged_data[numeric_columns].skew()
skewness = skewness[skewness > 1]
skewness

# apply power transformer to columns with skewness greater than 1
power_transformer = PowerTransformer()
merged_data[skewness.index] = power_transformer.fit_transform(merged_data[skewness.index])

# apply standard scaler to all numeric columns
scaler = StandardScaler()
merged_data[numeric_columns] = scaler.fit_transform(merged_data[numeric_columns])

# perform correlation matrix again
correlation_matrix2 = merged_data[numeric_columns].corr()
correlation_matrix2 = correlation_matrix2.round(2)
correlation_matrix2

# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix2, annot=True, cmap='coolwarm', fmt=".1f")
plt.title('Correlation Matrix')
plt.show()







In [ ]:
# minus correlation matrix from correlation matrix2
correlation_matrix_diff = correlation_matrix - correlation_matrix2
correlation_matrix_diff

# Create a visualisation of correlation matrix using seaborn heatmap with negatives being reds and positives being blues
plt.figure(figsize=(16, 6))
sns.heatmap(correlation_matrix_diff, annot=True, cmap='coolwarm', fmt=".1f")
plt.title('Correlation Matrix')
plt.show()


In [ ]:
# Compare the correlation matrix before and after the transformations
# create a new column transformer
column_transformer = ColumnTransformer(transformers=[('power_transformer', PowerTransformer(), skewness.index), ('standard_scaler', StandardScaler(), numeric_columns)])

# create a new pipeline
pipeline = Pipeline(steps=[('column_transformer', column_transformer)])

# fit and transform the pipeline
merged_data_transformed = pipeline.fit_transform(merged_data)

# convert transformed data to a dataframe
merged_data_transformed = pd.DataFrame(merged_data_transformed, columns=numeric_columns)
# perform correlation matrix on transformed data

correlation_matrix_transformed = merged_data_transformed.corr()
correlation_matrix_transformed = correlation_matrix_transformed.round(2)
correlation_matrix_transformed



In [ ]:
# preform a linear regression on the data to predict the pollution level
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

# create a list of numeric columns
numeric_columns = merged_data.select_dtypes(include=['float64', 'int64']).columns
# drop WS_Latitude, WS_Longitude from numeric_columns
numeric_columns = numeric_columns.drop(['WS_Latitude', 'WS_Longitude'])

# split data into features and target
X = merged_data[numeric_columns]
y = merged_data['NO2 AQI']

# split data into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# create a linear regression model
model = LinearRegression()
model.fit(X_train, y_train)

# predict the target using the model
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
mse

# save the model
import joblib

joblib.dump(model, 'Outputs/Linear_Regression_Model.pkl')

mse



In [ ]:
# load the model
model = joblib.load('Outputs/Linear_Regression_Model.pkl')

# predict the target using the model
y_pred = model.predict(X_test)

# calculate the mean squared error
mse = mean_squared_error(y_test, y_pred)
mse

# create a dataframe of the coefficients
coefficients = pd.DataFrame(model.coef_, index=numeric_columns, columns=['Coefficient'])
coefficients

# create a dataframe of the intercept
intercept = pd.DataFrame([model.intercept_], index=['Intercept'], columns=['Value'])
intercept

# save the coefficients and intercept to csv
coefficients.to_csv('Outputs/Coefficients.csv')
intercept.to_csv('Outputs/Intercept.csv')